In [1]:
import pickle as pkl
from utils.color_generator import ColorGenerator
import hashlib
from copy import copy
from dataclasses import dataclass

import cv2
from tqdm import tqdm
import mxnet as mx
import numpy as np
from gluoncv import model_zoo, data, utils
from gluoncv.data.transforms.pose import detector_to_alpha_pose, heatmap_to_coord_alpha_pose

gpu = mx.gpu()
pose_net = model_zoo.get_model('alpha_pose_resnet101_v1b_coco', pretrained=True, ctx=gpu)

/home/rafa/anaconda3/envs/sportstrack/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '
[20:10:38] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8600 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [2]:
with open('clustered_sequences.pkl', 'rb') as file:
    clustered_sequences = pkl.load(file)

In [3]:
from dataset.sequence import SkeletonSeq

with open('skeleton_sequences.pkl','rb') as file:
    skeleton_clustered_sequences = pkl.load(file)

for seq in tqdm(clustered_sequences[len(skeleton_clustered_sequences):]):
    gt = seq.gt
    start_frame = gt['frame'].min()
    end_frame = gt['frame'].max()
    is_in_frames = ((gt['frame'] >= start_frame) & (gt['frame'] <= end_frame))
    skeletons_seq = []
    confidence_seq = []
    classes_seq = []
    bboxes_seq = []
    scores_seq = []
    person_ids_seq = []

    for frame in range(start_frame, end_frame + 1):
        img = cv2.imread(seq.name_pattern.format(frame=frame))
        labels = seq.gt[seq.gt['frame'] == frame].to_records()
        bboxes = np.array([[label['x1'], label['y1'], label['x1'] + label['w'], label['y1'] + label['h']] for label in labels],dtype=np.float64)
        scores = mx.nd.array(np.array([np.ones(bboxes.shape[0])[:, None]]))
        classes= mx.nd.array(np.array([np.zeros(bboxes.shape[0])[:, None]]), ctx=gpu)
        bboxes = mx.nd.array(np.array([bboxes[:]]), ctx=gpu)
        person_ids = mx.nd.array(np.array([label['person_id'] for label in labels]))
        pose_input, upscale_bbox =  detector_to_alpha_pose(img, classes, scores, bboxes, ctx=gpu)
        pose_input = pose_input.copyto(gpu)
        predicted_heatmap = pose_net(pose_input)
        upscale_bbox = mx.nd.array(upscale_bbox, ctx=gpu)
        pred_coords, confidence = heatmap_to_coord_alpha_pose(predicted_heatmap, upscale_bbox, ctx=gpu)
        assert bboxes.shape[1] == pred_coords.shape[0]
        skeletons_seq.append(pred_coords.copyto(mx.cpu()).asnumpy())
        confidence_seq.append(confidence.copyto(mx.cpu()).asnumpy())
        classes_seq.append(classes.copyto(mx.cpu()).asnumpy())
        bboxes_seq.append(bboxes.copyto(mx.cpu()).asnumpy())
        scores_seq.append(scores.copyto(mx.cpu()).asnumpy())
        person_ids_seq.append(person_ids.copyto(mx.cpu()).asnumpy())

    skeleton_clustered_seq = SkeletonSeq(
        seq.name_pattern,
        start_frame,
        end_frame,
        skeletons_seq,
        confidence_seq,
        classes_seq,
        bboxes_seq,
        scores_seq,
        person_ids_seq
    )
    skeleton_clustered_sequences.append(skeleton_clustered_seq)
    with open('skeleton_sequences.pkl','wb') as file:
        pkl.dump(skeleton_clustered_sequences, file)
    # for frame in tqdm(range(start_frame, end_frame + 1)):
    #     img = cv2.imread(seq.name_pattern.format(frame=frame))
    #     ax = utils.viz.cv_plot_keypoints(img, skeletons_seq[frame-start_frame], confidence_seq[frame-start_frame],
    #                                      classes_seq[frame-start_frame], bboxes_seq[frame-start_frame], scores_seq[frame-start_frame],
    #                                      box_thresh=0.5, keypoint_thresh=0.2)
#                                      # labels=det_labels, colors=colors)
#         cv2.imshow('frame', ax)
#         cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)


  0%|          | 0/911 [00:00<?, ?it/s][20:10:41] ../src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
[20:10:48] ../src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
  0%|          | 2/911 [00:45<5:52:43, 23.28s/it][20:11:32] ../src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
[20:11:46] ../src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 